# Helsinki Property Visualization

This notebook visualizes property data for Helsinki, using data from the National Land Survey of Finland. It demonstrates how to load, filter, and plot geospatial data with a map background.

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import contextily as cx
import fiona
import duckdb
from shapely import wkt
from IPython.display import display, Markdown

# --- Configuration ---
DB_PATH = os.path.join('..', 'data', 'real_estate.duckdb')
PROPERTIES_TABLE_NAME = 'helsinki_properties'
gpkg_path = os.path.join('..', 'data', 'open', 'kiinteistorekisterikartta.gpkg')
municipality_zip_path = os.path.join('..', 'data', 'open', 'TietoaKuntajaosta_2025_10k.zip')
helsinki_municipality_name = 'Helsinki'

## 1. Load Helsinki Data

In [ ]:
def load_from_db():
    with duckdb.connect(DB_PATH) as con:
        tables = con.execute("SHOW TABLES;").fetchdf()
        if PROPERTIES_TABLE_NAME not in tables['name'].values:
            return None
        df = con.execute(f'SELECT * FROM {PROPERTIES_TABLE_NAME}').fetchdf()
    df['geometry'] = df['geometry_wkt'].apply(wkt.loads)
    return gpd.GeoDataFrame(df, geometry='geometry')

def load_from_file():
    with fiona.open(f'zip://{municipality_zip_path}') as collection:
        municipalities = gpd.GeoDataFrame.from_features(collection, crs=collection.crs)
    helsinki = municipalities[municipalities['namefin'] == helsinki_municipality_name]
    helsinki_boundary = helsinki.geometry.union_all()
    
    gdf_kiinteisto = gpd.read_file(gpkg_path, layer='KiinteistorajanSijaintitiedot')
    return gdf_kiinteisto[gdf_kiinteisto.within(helsinki_boundary)]

helsinki_properties = load_from_db()

if helsinki_properties is None:
    display(Markdown('## ⚠️ Could not load data from the database. Falling back to loading from source files.
For better performance, run `python prepare_geospatial_data.py` first.'))
else:
    display(Markdown('## ✅ Successfully loaded data from the database.'))

display(Markdown(f'Loaded **{len(helsinki_properties)}** features for Helsinki.'))

## 2. Visualize Two Random Properties

In [ ]:
def plot_property(gdf, title):
    property_to_plot = gdf.sample(n=1, random_state=None)
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    property_to_plot.to_crs(epsg=3857).plot(ax=ax, alpha=0.5, edgecolor='k')
    cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
    ax.set_title(title)
    ax.set_axis_off()
    plt.show()

if 'helsinki_properties' in locals() and not helsinki_properties.empty:
    plot_property(helsinki_properties, 'Random Property 1')
    plot_property(helsinki_properties, 'Random Property 2')

## 3. Visualize Properties within a Random Postal Code Area

In [ ]:
if 'helsinki_properties' in locals() and not helsinki_properties.empty:
    # This is a placeholder for actual postal code data
    # In a real scenario, you would load a postal code shapefile for Helsinki
    postal_codes_gdf = gpd.read_file(gpd.datasets.get_path('nybb')) # Using a sample dataset for now
    random_postal_code = postal_codes_gdf.sample(n=1, random_state=42)
    
    # Find properties within the random postal code area
    properties_in_postal_code = helsinki_properties[helsinki_properties.within(random_postal_code.union_all())]
    
    if not properties_in_postal_code.empty:
        fig, ax = plt.subplots(1, 1, figsize=(12, 12))
        properties_in_postal_code.to_crs(epsg=3857).plot(ax=ax, alpha=0.7, edgecolor='blue', facecolor='lightblue')
        random_postal_code.to_crs(epsg=3857).plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)
        cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)
        ax.set_title(f'Properties within Postal Code: {random_postal_code.BoroName.iloc[0]}')
        ax.set_axis_off()
        plt.show()
    else:
        display(Markdown('No properties found in the selected random postal code area.'))

## ✅ Verification Cell

In [ ]:
display(Markdown('## ✅ Notebook Executed Successfully'))